In [8]:
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import os

class TomatoMultiLabelDataset(Dataset):
    def __init__(self, csv_file, root_dir, subset="train", transform=None):
        """
        Args:
            csv_file (str): Path to the CSV file.
            root_dir (str): Root directory containing train, val, and test folders.
            subset (str): One of 'train', 'val', or 'test'.
            transform (callable, optional): Transformations to apply to images.
        """
        self.df = pd.read_csv(csv_file)
        # Ensure all missing label columns are filled with 0 (assuming 0 means absence)
        self.df.fillna(0, inplace=True)

        # Filter dataset based on 'train', 'val', or 'test'
        self.df = self.df[self.df["dataset"] == subset]

        self.root_dir = root_dir
        self.transform = transform

        # Get label names (all columns after 'path')
        self.label_columns = self.df.columns[3:-1]  # Exclude 'dataset', 'filename', 'path', and 'SUM'

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        # Get image path
        img_name = self.df.iloc[idx]["filename"]
        img_folder = self.df.iloc[idx]["dataset"]  # 'train', 'val', or 'test'
        img_path = os.path.join(self.root_dir, img_folder, img_name)

        # Load image
        image = Image.open(img_path).convert("RGB")

        # Apply transformations
        if self.transform:
            image = self.transform(image)

        # Load labels (convert to tensor)
        labels = torch.tensor(self.df.iloc[idx][self.label_columns].values.astype(float), dtype=torch.float32)

        return image, labels

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to match model input
    transforms.ToTensor(),          # Convert to tensor
])

# Create dataset
csv_path = "Tomato-Village-main/Variant-b(MultiLabel Classification)/Multi-Label dataset - with augmented.csv"
root_dir = "Tomato-Village-main/Variant-b(MultiLabel Classification)/"

train_dataset = TomatoMultiLabelDataset(csv_file=csv_path, root_dir=root_dir, subset="train", transform=transform)

# Create DataLoader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Test the dataset
for images, labels in train_loader:
    print(images.shape)  # Should be (batch_size, 3, 224, 224)
    print(labels.shape)  # Should be (batch_size, num_classes)
    break


torch.Size([32, 3, 224, 224])
torch.Size([32, 8])


In [9]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import timm
from torchvision.models import vit_b_16


In [15]:
# Define transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # ViT input size
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),  # Normalize for ViT
])

# Dataset paths
csv_path = "Tomato-Village-main/Variant-b(MultiLabel Classification)/Multi-Label dataset - with augmented.csv"
root_dir = "Tomato-Village-main/Variant-b(MultiLabel Classification)/"

# Load datasets
train_dataset = TomatoMultiLabelDataset(csv_file=csv_path, root_dir=root_dir, subset="train", transform=transform)
val_dataset = TomatoMultiLabelDataset(csv_file=csv_path, root_dir=root_dir, subset="val", transform=transform)
test_dataset = TomatoMultiLabelDataset(csv_file=csv_path, root_dir=root_dir, subset="test", transform=transform)

# Data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)



In [11]:
# Load pretrained ViT
num_labels = len(train_dataset.label_columns)  # Get number of labels (e.g., 8)

model = vit_b_16(weights="IMAGENET1K_V1")  # Load a pretrained model
model.heads.head = nn.Linear(model.heads.head.in_features, num_labels)  # Modify last layer


In [12]:
# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(device)
# Loss function & optimizer
criterion = nn.BCEWithLogitsLoss()  # Suitable for multilabel classification
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)


cuda


In [13]:
# Training function
def train_model(model, train_loader, val_loader, criterion, optimizer, epochs=10):
    for epoch in range(epochs):
        model.train()
        train_loss = 0.0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)  # Logits
            loss = criterion(outputs, labels)  # Compute loss
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        avg_train_loss = train_loss / len(train_loader)

        # Validation phase
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

        avg_val_loss = val_loss / len(val_loader)

        print(f"Epoch [{epoch+1}/{epochs}] | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")

# Train for 10 epochs
train_model(model, train_loader, val_loader, criterion, optimizer, epochs=10)


Epoch [1/10] | Train Loss: 0.2277 | Val Loss: 0.1724
Epoch [2/10] | Train Loss: 0.1205 | Val Loss: 0.1307
Epoch [3/10] | Train Loss: 0.0719 | Val Loss: 0.1640
Epoch [4/10] | Train Loss: 0.0496 | Val Loss: 0.1670
Epoch [5/10] | Train Loss: 0.0400 | Val Loss: 0.1784
Epoch [6/10] | Train Loss: 0.0320 | Val Loss: 0.1406
Epoch [7/10] | Train Loss: 0.0250 | Val Loss: 0.1602
Epoch [8/10] | Train Loss: 0.0228 | Val Loss: 0.2031
Epoch [9/10] | Train Loss: 0.0255 | Val Loss: 0.1700
Epoch [10/10] | Train Loss: 0.0202 | Val Loss: 0.1671


In [17]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

def evaluate_model(model, test_loader, class_names):
    model.eval()  # Set to evaluation mode
    all_labels = []
    all_preds = []

    with torch.no_grad():  # No gradients needed
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            outputs = torch.sigmoid(outputs)  # Convert logits to probabilities
            preds = (outputs > 0.7).int()  # Convert probabilities to binary (0 or 1)

            all_labels.append(labels.cpu().numpy())  # Store true labels
            all_preds.append(preds.cpu().numpy())  # Store predictions

    # Convert lists to numpy arrays
    all_labels = np.concatenate(all_labels, axis=0)
    all_preds = np.concatenate(all_preds, axis=0)

    # Compute metrics for each class
    metrics = {}
    for i, class_name in enumerate(class_names):
        acc = accuracy_score(all_labels[:, i], all_preds[:, i])
        precision = precision_score(all_labels[:, i], all_preds[:, i], zero_division=0)
        recall = recall_score(all_labels[:, i], all_preds[:, i], zero_division=0)
        f1 = f1_score(all_labels[:, i], all_preds[:, i], zero_division=0)

        metrics[class_name] = {
            "Accuracy": acc,
            "Precision": precision,
            "Recall": recall,
            "F1-score": f1,
        }

    return metrics

# Define class names (based on CSV headers)
class_names = ["Early blight", "Healthy", "Late blight", "Leaf Miner", 
               "Magnesium Deficiency", "Nitrogen Deficiency", 
               "Potassium Deficiency", "Spotted Wilt Virus"]

# Evaluate model
metrics = evaluate_model(model, test_loader, class_names)

# Print results
for category, scores in metrics.items():
    print(f"\nCategory: {category}")
    for metric, value in scores.items():
        print(f"  {metric}: {value:.4f}")



Category: Early blight
  Accuracy: 0.9486
  Precision: 0.8780
  Recall: 0.5902
  F1-score: 0.7059

Category: Healthy
  Accuracy: 0.9692
  Precision: 0.6111
  Recall: 0.5000
  F1-score: 0.5500

Category: Late blight
  Accuracy: 0.9572
  Precision: 0.9268
  Recall: 0.8000
  F1-score: 0.8588

Category: Leaf Miner
  Accuracy: 0.8921
  Precision: 0.8786
  Recall: 0.8265
  F1-score: 0.8518

Category: Magnesium Deficiency
  Accuracy: 0.9469
  Precision: 0.9320
  Recall: 0.8671
  F1-score: 0.8984

Category: Nitrogen Deficiency
  Accuracy: 0.9675
  Precision: 0.8444
  Recall: 0.7600
  F1-score: 0.8000

Category: Potassium Deficiency
  Accuracy: 0.9914
  Precision: 1.0000
  Recall: 0.5000
  F1-score: 0.6667

Category: Spotted Wilt Virus
  Accuracy: 0.9281
  Precision: 0.7500
  Recall: 0.8438
  F1-score: 0.7941
